# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from config import api_key
import json

In [4]:
# Import API key
#from api_keys_copy import weather_api_key

In [5]:
api_key = "9271bf5b20e182157ecf5ac650f5f6c1"

In [6]:
# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

In [7]:
# Output File (CSV)
output_data_file = "output_data_cities.csv"

#Read csv file
output_data = pd.read_csv(output_data_file)

In [8]:
output_data.shape

(548, 10)

In [9]:
output_data.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [10]:
# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [11]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []


# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=60)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=60)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

53

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [12]:
# API url
base_url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"


In [13]:
# Create lists

humidity = []
latitude = []
longitude = []
cloudiness = []
temperature = []
wind_speed = []
date = []
city_name = []
country = []

In [14]:
record_counter = 0
set_counter = 0
units = "imperial"

for city in cities:
    
    # Query url
    query_url = f"{base_url}appid={api_key}&q={city}&units={units}"    
    if record_counter < 50:
        record_counter += 1
    else:
        set_counter +=1
        record_counter = 0
        time.sleep(90)
        
    print(f"Processing record {record_counter} of set {set_counter} | {city}")
    
    response = requests.get(query_url).json()
    
    try:
        
        humidity.append(response['main']['humidity'])
        latitude.append(response['coord']['lat'])
        longitude.append(response['coord']['long'])
        cloudiness.append(response['cloud']['long'])
        temperature.append(response['main']['temp_max'])
        wind_speed.append(response['wind']['speed'])
        date.append(response['dt'])
        city_name.append(response['city'])
        country.append(response['sys']['country'])
      
    except:
        print(f"City {city} not Found...")
        pass

KeyboardInterrupt: 

In [ ]:
response.status_code

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
# Combine collected data list to dataframe
weather_df = pd.DataFrame({"City":city_name,
                           "Latitude":latitude,
                           "Longitude":longitude,
                           "Max_Temp":temperature,
                           "Humidity":humidity,
                           "Cloudiness":cloudiness,
                           "Wind_Speed":wind_speed,
                           "Date":date,
                           "Country":country
                          })

weather_df.info()

In [ ]:
# Weather dataframe to csv
weather_df.to_csv(output_data_file)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
humid_df = weather_df.loc[weather_df.Humidity < 101]

In [ ]:
#  Get the indices of cities that have humidity over 100%.
over_hundred = []
over_hundred = weather_df.loc[weather_df.Humidity > 100].index
over_hundred

In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.

# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".

In [ ]:
clean_city_data = weather_df.drop[weather_df.index[over_hundred], inplace=False]

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
clean_city_data.plot(kind=scatter, x='latitude', y='max_temp', color='blue')
plt.title('Latitude vs. Max Temperature')
plt.xlabel('Latitude')
plt.ylabel('Max Temperature (F)')
plt.grid()
plt.savefig("plot_data/lat_temp.png")

## Latitude vs. Humidity Plot

In [ ]:
clean_city_data.plot(kind=scatter, x='latitude', y='humidity', color='blue')
plt.title('Latitude vs. Humidity')
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.grid()
plt.savefig("plot_data/lat_humidity.png")

## Latitude vs. Cloudiness Plot

In [ ]:
clean_city_data.plot(kind=scatter, x='latitude', y='cloudiness', color='blue')
plt.title('Latitude vs. Cloudiness')
plt.xlabel('Latitude')
plt.ylabel('Cloudiness')
plt.grid()
plt.savefig("plot_data/lat_cloudiness.png")

## Latitude vs. Wind Speed Plot

In [ ]:
clean_city_data.plot(kind=scatter, x='latitude', y='wind_speed', color='blue')
plt.title('Latitude vs. Wind Speed')
plt.xlabel('Latitude')
plt.ylabel('Wind Speed')
plt.grid()
plt.savefig("plot_data/lat_windspeed.png")

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
northern = clean_city_data.loc[clean_city_data.lat >=0]
northern.info()

In [ ]:
southern = clean_city_data.loc[clean_city_data.lat <0]
southern.info()

In [ ]:
x_values = northern.latitude
y_values = northern.max_temp
plt.scatter(x_values,y_values)

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))


plt.plot(x_values,regress_values,"r")
plt.annotate(line_eq,(40,62),fontsize=15,color="red")
plt.title =("Northern Hemisphere: Latitude vs. Max Temperature")
plt.xlabel('Latitude')
plt.ylabel('Max Temperature (F)')
plt.grid()
plt.savefig("plot_data/northern_lat_temp.png")

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_values = southern.latitude
y_values = southern.max_temp
plt.scatter(x_values,y_values)

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))


plt.plot(x_values,regress_values,"r")
plt.annotate(line_eq,(40,62),fontsize=15,color="red")
plt.title =("Southern Hemisphere: Latitude vs. Max Temperature")
plt.xlabel('Latitude')
plt.ylabel('Max Temperature (F)')
plt.grid()
plt.savefig("plot_data/southern_lat_temp.png")

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_values = northern.latitude
y_values = northern.humidity
plt.scatter(x_values,y_values)

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))


plt.plot(x_values,regress_values,"r")
plt.annotate(line_eq,(40,62),fontsize=15,color="red")
plt.title =("Northern Hemisphere: Latitude vs. Humidity (%)")
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.grid()
plt.savefig("plot_data/northern_lat_humidity.png")

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_values = southern.latitude
y_values = southern.humidity
plt.scatter(x_values,y_values)

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))


plt.plot(x_values,regress_values,"r")
plt.annotate(line_eq,(40,62),fontsize=15,color="red")
plt.title =("Southern Hemisphere: Latitude vs. Humidity (%)")
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.grid()
plt.savefig("plot_data/southern_lat_humidity.png")

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_values = northern.latitude
y_values = northern.cloudiness
plt.scatter(x_values,y_values)

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))


plt.plot(x_values,regress_values,"r")
plt.annotate(line_eq,(40,62),fontsize=15,color="red")
plt.title =("Northern Hemisphere: Latitude vs. Cloudiness (%)")
plt.xlabel('Latitude')
plt.ylabel('Cloudiness')
plt.grid()
plt.savefig("plot_data/northern_lat_cloud.png")

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_values = southern.latitude
y_values = southern.cloudiness
plt.scatter(x_values,y_values)

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))


plt.plot(x_values,regress_values,"r")
plt.annotate(line_eq,(40,62),fontsize=15,color="red")
plt.title =("Southern Hemisphere: Latitude vs. Cloudiness (%)")
plt.xlabel('Latitude')
plt.ylabel('Cloudiness')
plt.grid()
plt.savefig("plot_data/southern_lat_cloud.png")

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_values = northern.latitude
y_values = northern.wind_speed
plt.scatter(x_values,y_values)

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))


plt.plot(x_values,regress_values,"r")
plt.annotate(line_eq,(40,62),fontsize=15,color="red")
plt.title =("Northern Hemisphere: Latitude vs. Wind Speed (mph)")
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
plt.grid()
plt.savefig("plot_data/northern_lat_windspeed.png")

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_values = southern.latitude
y_values = southern.wind_speed
plt.scatter(x_values,y_values)

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))


plt.plot(x_values,regress_values,"r")
plt.annotate(line_eq,(40,62),fontsize=15,color="red")
plt.title =("Southern Hemisphere: Latitude vs. Wind Speed (mph)")
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
plt.grid()
plt.savefig("plot_data/southern_lat_windspeed.png")